In [9]:
import requests
import time
import pandas as pd
import mysql.connector  
import numpy as np

all_data = []
page = 1

while page <= 5: 
    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=inr&per_page=250&page={page}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if not data: # Break if we reach an empty page
            break
        all_data.extend(data)
        print(f"Page {page} fetched successfully ({len(all_data)} coins total)")
        page += 1 
        time.sleep(10) # Increase sleep to 10s for free tier safety
        
    elif response.status_code == 429:
        print(f"Rate limited on page {page}. Waiting 60 seconds...")
        time.sleep(60) # If blocked, wait a full minute
        
    else:
        print(f"Error {response.status_code} at page {page}. Stopping.")
        break



# --- 2. Process into Records (Same as your code) ---
records = [ {
        'id': i['id'],
        'market_cap_rank': i['market_cap_rank'],
        'symbol': i['symbol'],
        'name': i['name'],
        'current_price': i['current_price'],
        'market_cap': i['market_cap'],
        'total_volume': i['total_volume'],
        'circulating_supply': i['circulating_supply'],
        'total_supply': i.get('total_supply'),
        'ath': i['ath'],
        'atl': i['atl'],
        'last_updated': i['last_updated']
} for i in all_data ]

df = pd.DataFrame(records)



Page 1 fetched successfully (250 coins total)
Page 2 fetched successfully (500 coins total)
Page 3 fetched successfully (750 coins total)
Rate limited on page 4. Waiting 60 seconds...
Page 4 fetched successfully (1000 coins total)
Page 5 fetched successfully (1250 coins total)


In [10]:
df

,id,market_cap_rank,symbol,name,current_price,market_cap,total_volume,circulating_supply,total_supply,ath,atl,last_updated
0,bitcoin,1,btc,Bitcoin,6.163697e+06,122987650473266,4.363807e+12,1.998765e+07,1.998765e+07,1.118701e+07,3.993420e+03,2026-02-11T14:09:22.534Z
1,ethereum,2,eth,Ethereum,1.797220e+05,21648624614097,1.796665e+12,1.206926e+08,1.206926e+08,4.319460e+05,2.813000e+01,2026-02-11T14:09:22.751Z
2,tether,3,usdt,Tether,9.073000e+01,16731313106103,6.679260e+12,1.843940e+11,1.898610e+11,1.055200e+02,3.686000e+01,2026-02-11T14:09:24.149Z
3,ripple,4,xrp,XRP,1.255900e+02,7654910708224,2.483195e+11,6.091732e+10,9.998572e+10,3.139900e+02,1.593430e-01,2026-02-11T14:09:23.968Z
4,binancecoin,5,bnb,BNB,5.476800e+04,7449223751949,1.217732e+11,1.363595e+08,1.363595e+08,1.214220e+05,2.580000e+00,2026-02-11T14:09:22.287Z
...,...,...,...,...,...,...,...,...,...,...,...,...
1245,tars-protocol,1246,tai,TARS AI,1.320000e+00,773281855,7.887450e+07,5.866813e+08,8.949961e+08,4.187000e+01,1.008000e+00,2026-02-11T14:11:02.650Z
1246,lumia,1247,lumia,Lumia,5.580000e+00,772169812,1.823697e+08,1.384960e+08,2.388889e+08,2.113100e+02,5.350000e+00,2026-02-11T14:11:03.131Z
1247,bitcoin-usd-btcfi,1248,btcusd,Bitcoin USD (BTCFi),9.066000e+01,771487801,1.722445e+06,8.507197e+06,8.507197e+06,1.015000e+02,7.936000e+01,2026-02-11T14:11:02.605Z
1248,pepper,1249,pepper,PEPPER,8.742200e-08,768842623,1.478463e+06,8.760094e+15,8.760094e+15,2.459000e-05,5.953400e-08,2026-02-11T14:11:02.982Z


In [11]:
crypto_df = pd.DataFrame(records) #Creating Dataframe
crypto_df=crypto_df.dropna() #droping null values
crypto_df['last_updated'] = pd.to_datetime(crypto_df['last_updated']) #Converting the 'last_updated' column to actual Datetime
crypto_df['date'] = crypto_df['last_updated'].dt.date
crypto_df.head()

,id,market_cap_rank,symbol,name,current_price,market_cap,total_volume,circulating_supply,total_supply,ath,atl,last_updated,date
0,bitcoin,1,btc,Bitcoin,6163697.00,122987650473266,4.363807e+12,1.998765e+07,1.998765e+07,11187013.00,3993.420000,2026-02-11 14:09:22.534000+00:00,2026-02-11
1,ethereum,2,eth,Ethereum,179722.00,21648624614097,1.796665e+12,1.206926e+08,1.206926e+08,431946.00,28.130000,2026-02-11 14:09:22.751000+00:00,2026-02-11
2,tether,3,usdt,Tether,90.73,16731313106103,6.679260e+12,1.843940e+11,1.898610e+11,105.52,36.860000,2026-02-11 14:09:24.149000+00:00,2026-02-11
3,ripple,4,xrp,XRP,125.59,7654910708224,2.483195e+11,6.091732e+10,9.998572e+10,313.99,0.159343,2026-02-11 14:09:23.968000+00:00,2026-02-11
4,binancecoin,5,bnb,BNB,54768.00,7449223751949,1.217732e+11,1.363595e+08,1.363595e+08,121422.00,2.580000,2026-02-11 14:09:22.287000+00:00,2026-02-11


In [12]:
#Converting to INR
exchange_rate = 87.50 #Fixing a INR value
crypto_df['current_price_inr'] = crypto_df['current_price'] * exchange_rate
crypto_df['current_price_inr'] = crypto_df['current_price_inr'].round(2) #rounding off
print(crypto_df[['name', 'current_price', 'current_price_inr']].head())

       name  current_price  current_price_inr
0   Bitcoin     6163697.00       5.393235e+08
1  Ethereum      179722.00       1.572568e+07
2    Tether          90.73       7.938880e+03
3       XRP         125.59       1.098912e+04
4       BNB       54768.00       4.792200e+06


In [20]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="jujo",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

In [ ]:
mycursor.execute("create database crypto_analysis")

In [21]:
mycursor.execute("use crypto_analysis")

In [3]:
# 2. CREATE TABLE (Ensure names here match the INSERT query)
mycursor.execute("""
   CREATE TABLE Cryptocurrencies (
        id VARCHAR(255) PRIMARY KEY,
        `rank` INT,
        symbol VARCHAR(50),
        name VARCHAR(255),
        current_price DECIMAL(18, 6),
        market_cap BIGINT,
        total_volume BIGINT,
        circulating_supply DECIMAL(38, 6),
        total_supply DECIMAL(38, 6),
        ath DECIMAL(18, 6),
        atl DECIMAL(18, 6),
        `date` DATE
      )
""")

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [19]:
#Insterting to the Table
import time
start = time.time() 

cols = ['id', 'market_cap_rank', 'symbol', 'name', 'current_price_inr', 'market_cap', 
        'total_volume', 'circulating_supply', 'total_supply', 'ath', 'atl', 'date',] # Dataframe Columns
crypto_df_clean = crypto_df[cols].dropna()  


data_to_insert = [tuple(x) for x in crypto_df_clean.values]

sql = """
INSERT INTO Cryptocurrencies (
    id, `rank`, symbol, name, current_price, market_cap, 
    total_volume, circulating_supply, total_supply, ath, atl, `date`
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
""" # Table columns

try:
    mycursor.executemany(sql, data_to_insert)
    mydb.commit() 
    total = mycursor.rowcount
    elapsed = time.time() - start
    print(f"\n✓ Successfully inserted {total} records!")
    
except Exception as e:
    mydb.rollback()
    print(f"Error: {e}")

Error: 1213 (40001): Deadlock found when trying to get lock; try restarting transaction


In [14]:

query_1 = "SELECT name, market_cap, current_price FROM Cryptocurrencies ORDER BY market_cap DESC LIMIT 3"

mycursor.execute(query_1)

top_3_results = mycursor.fetchall()

print("Top 3 Cryptocurrencies by Market Cap:")
for row in top_3_results:
    print(f"Name: {row[0]} | Market Cap: {row[1]} | Price: {row[2]}")

Top 3 Cryptocurrencies by Market Cap:
Name: Bitcoin | Market Cap: 122987650473266 | Price: 539323487.500000
Name: Ethereum | Market Cap: 21648624614097 | Price: 15725675.000000
Name: Tether | Market Cap: 16731313106103 | Price: 7938.880000


In [ ]:
#cryptocurrencies #Query 2

In [15]:
query2 = """
SELECT name, 
        circulating_supply, 
        total_supply, 
       (circulating_supply / total_supply) * 100 AS supply_percentage
FROM Cryptocurrencies 
WHERE (circulating_supply / total_supply) > 0.9
ORDER BY supply_percentage DESC
"""

mycursor.execute(query2)
results = mycursor.fetchall()

print(f"{'Name':<15} | {'Supply %':<10}")
print("-" * 30)
for row in results:
    print(f"{row[0]:<15} | {row[3]:.2f}%")

Name            | Supply %  
------------------------------
Resolv USR      | 104.57%
Monerium EUR emoney | 101.52%
StandX DUSD     | 100.83%
Metronome Synth USD | 100.67%
Ondo U.S. Dollar Token | 100.57%
Aster USDF      | 100.03%
Yield Optimizer USD | 100.01%
BOLD            | 100.00%
OpenEden OpenDollar | 100.00%
USDA            | 100.00%
WAR             | 100.00%
PunkStrategy    | 100.00%
USDai           | 100.00%
TRON            | 100.00%
Nietzschean Penguin | 100.00%
pippin          | 100.00%
0xy             | 100.00%
4               | 100.00%
A7A5            | 100.00%
Act I The AI Prophecy | 100.00%
Aegis YUSD      | 100.00%
affine          | 100.00%
AgentFun.AI     | 100.00%
AUSD            | 100.00%
AI Companions   | 100.00%
AI Rig Complex  | 100.00%
AIntivirus      | 100.00%
Alchemix USD    | 100.00%
Alloy Tether    | 100.00%
Alphabet Class A (Ondo Tokenized Stock) | 100.00%
AMO Coin        | 100.00%
Anchored Coins AEUR | 100.00%
ANDY ETH        | 100.00%
ANDY70B         | 100

In [13]:
#cryptocurrencies #Query 3  coins within 10% of their ATH

In [28]:

query3 = """
SELECT name, current_price, ath, 
       ((current_price / ath) * 100) AS percent_of_ath
FROM Cryptocurrencies 
WHERE current_price >= (ath * 0.9)
ORDER BY percent_of_ath DESC
"""

mycursor.execute(query3)
results = mycursor.fetchall()

for row in results:
    print(f" Name: {row[0]} ")

 Name: MimboGameGroup 
 Name: DIAMOND TOKEN 💎 
 Name: Yochi 
 Name: Mu Digital AZND 
 Name: Fidelity Digital Dollar 
 Name: Trevee Plasma USD 
 Name: AIntivirus 
 Name: USDa 
 Name: Main Street Yield 
 Name: Onchain Yield Coin 
 Name: Midas mRe7YIELD 
 Name: Circle USYC 
 Name: iShares MSCI EAFE ETF (Ondo Tokenized ETF) 
 Name: Midas mHYPER 
 Name: VCRED 
 Name: Midas mMEV 
 Name: Midas msyrupUSDp 
 Name: Yield Optimizer USD 
 Name: USTBL 
 Name: Spiko US T-Bills Money Market Fund 
 Name: OUSG 
 Name: BFUSD 
 Name: Midas mTBILL 
 Name: GHO 
 Name: Midas mAPOLLO 
 Name: Liquity USD 
 Name: Worldwide USD 
 Name: USAT 
 Name: Resolv USR 
 Name: Play 
 Name: BOLD 
 Name: Ripple USD 
 Name: Frax USD 
 Name: Ondo U.S. Dollar Token 
 Name: Freedom Dollar 
 Name: Hylo USD 
 Name: AUSD 
 Name: Stable Coin 
 Name: YLDS 
 Name: USDS 
 Name: GAIB AID 
 Name: USD1 
 Name: USDD 
 Name: PayPal USD 
 Name: Bitcoin Limited Edition 
 Name: Austin Capitals 
 Name: iShares TIPS Bond ETF (Ondo Tokenized ET

In [17]:
# Query to calculate the average rank for high-volume coins
query4 = """
SELECT AVG(`rank`) as average_rank, 
    COUNT(*) as coin_count
    FROM Cryptocurrencies
    WHERE total_volume > 1000000000
"""

mycursor.execute(query4)
result = mycursor.fetchone()

print(f"Average Market Cap Rank: {round(result[0], 2)}")
print(f"Number of coins with >$1B Volume: {result[1]}")

Average Market Cap Rank: 310.37
Number of coins with >$1B Volume: 255


In [ ]:
#cryptocurrencies #Query 5 single most recently updated record

In [18]:
query_5 = "SELECT * FROM Cryptocurrencies ORDER BY 'date' DESC LIMIT 1"

mycursor.execute(query_5)
result = mycursor.fetchone()

# Extract column names to make the output readable
columns = [i[0] for i in mycursor.description]

if result:
    print("Most Recently Updated Coin:")
    for col, val in zip(columns, result):
        print(f"{col}: {val}")
else:
    print("No data found in the table.")

Most Recently Updated Coin:
id: 0x
rank: 295
symbol: zrx
name: 0x Protocol
current_price: 817.250000
market_cap: 7907126489
total_volume: 818114040
circulating_supply: 848396562.906836
total_supply: 1000000000.000000
ath: 179.790000
atl: 8.090000
date: 2026-02-11
